<a href="https://colab.research.google.com/github/nxxk23/AI-Engineer/blob/main/fraud/get_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip -q install langchain langchain_huggingface langchain_community  gradio transformers elasticsearch gradio_calendar

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 543.1/543.1 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 95.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 74.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 

In [10]:
import os
from datetime import datetime
from getpass import getpass
import requests
from huggingface_hub import InferenceClient
from elasticsearch import Elasticsearch
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer
import gradio as gr
import json
from gradio_calendar import Calendar
import logging
import threading
import time
import re


## LLM credentials
username_llm = "manageai2024"
passwords_llm  = "ManageAI@2024"

# Elasticsearch credentials
user = "natthaphol.po"
passwords = "cnEM5CeFrG"

tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
model = AutoModel.from_pretrained('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
es = Elasticsearch(
    hosts=[{'host': 'elk.manageai.co.th', 'port': 443, 'scheme': 'https'}],
    basic_auth=(user, passwords)
)
3
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

def rag_encode(text):
    encoded_input = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**encoded_input)
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
    sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
    return sentence_embeddings.squeeze().tolist()

def search_exact_match_score(query_text, index_name):
    query_vector = rag_encode(query_text)
    search_query = {
        "size": 10,
        "_source": ["icd10", "total_average_price"],  # Include only required fields
        "query": {
            "script_score": {
                "query": {
                    "match_all": {}
                },
                "script": {
                    "source": "cosineSimilarity(params.query_vector, 'embedding') + 1.0",
                    "params": {
                        "query_vector": query_vector
                    }
                }
            }
        }
    }

    try:
        response = es.search(index=index_name, body=search_query, timeout='120s')
        exact_score_matches = [
            {
                "_source": hit["_source"]
            }
            for hit in response['hits']['hits'] if hit["_score"] == 2.0
        ]

        if exact_score_matches:
            return exact_score_matches
        else:
            print("No documents found with score 2.0.")
            return None

    except Exception as e:
        print(f"Error performing search: {e}")
        return None

def search_similar_vectors(query_text, index_name):
    query_vector = rag_encode(query_text)

    search_query = {
        "size": 1000,
        "_source": [
            "simb_billing_code", "local_billing_name", "item_id", "item_name",
            "average_initial_price", "order_count", "icd10", "range"
        ],
        "query": {
            "script_score": {
                "query": {
                    "match_all": {}
                },
                "script": {
                    "source": "cosineSimilarity(params.query_vector, 'embedding') + 1.0",
                    "params": {
                        "query_vector": query_vector
                    }
                }
            }
        }
    }

    try:
        response = es.search(index=index_name, body=search_query, timeout='120s')
        results = []
        for hit in response['hits']['hits']:
            score = hit["_score"]
            if score == 2.0:
                result = {
                    "_index": hit["_index"],
                    "_id": hit["_id"],
                    "_score": score,
                    "_source": hit["_source"]
                }
                results.append(result)

        if results:
            source_data = [hit["_source"] for hit in results]
            df = pd.DataFrame(source_data)
            return df
        else:
            print("No results with score 2.0 found.")
            return None

    except Exception as e:
        print(f"Error performing search: {e}")
        return None


# กำหนดตัวแปร Global สำหรับเก็บข้อมูลที่อัปโหลด
uploaded_data = None
uploaded_simb = None
uploaded_diag = None
sample_cids = []
diagnosis_mapping = {}

# ฟังก์ชันที่โหลดไฟล์และดึงข้อมูล sample_cids
def load_files(data_file, simb_file, diag_file):
    global uploaded_data, uploaded_simb, uploaded_diag, sample_cids, diagnosis_mapping

    try:
        # โหลดไฟล์ CSV
        if data_file:
            uploaded_data = pd.read_csv(data_file.name)
        if simb_file:
            uploaded_simb = pd.read_csv(simb_file.name)
        if diag_file:
            uploaded_diag = pd.read_csv(diag_file.name)

        # สร้าง diagnosis_mapping จาก diag_file
        if uploaded_diag is not None:
            diagnosis_mapping = {
                row['ICD-10 Code']: row['Description'].strip()
                for _, row in uploaded_diag.iterrows()
            }
        else:
            diagnosis_mapping = {}

        # ตรวจสอบว่า data_file มีข้อมูล
        if uploaded_data is not None:
            # ดึง sample_cids ที่ไม่ใช่การติดตาม (is_follow_up == 'N')
            sample_cids = uploaded_data[uploaded_data['is_follow_up'] == 'N']['cid'].sample(5).tolist()
            print(f"Sample CIDs: {sample_cids}")  # ตรวจสอบค่าของ sample_cids
        else:
            sample_cids = []

        # คืนค่าผลลัพธ์และรายการ CID ตัวอย่าง
        return "ไฟล์ถูกอัปโหลดสำเร็จ!", gr.update(choices=sample_cids), gr.update(choices=list(diagnosis_mapping.values()))

    except Exception as e:
        # คืนค่าข้อความแสดงข้อผิดพลาด
        return f"เกิดข้อผิดพลาดในการอัปโหลดไฟล์: {str(e)}", gr.update(choices=[]), gr.update(choices=[])

# ฟังก์ชันสำหรับใช้งานไฟล์ที่อัปโหลด
def analyze_uploaded_data():
    global uploaded_data, uploaded_simb, uploaded_diag

    if uploaded_data is None or uploaded_simb is None or uploaded_diag is None:
        return "กรุณาอัปโหลดไฟล์ทั้งหมดก่อน", None

    # ตัวอย่างการประมวลผลข้อมูล (เช่น นับจำนวนแถวในแต่ละไฟล์)
    data_summary = {
        "data.csv": len(uploaded_data),
        "simblist.csv": len(uploaded_simb),
        "diaglist.csv": len(uploaded_diag),
    }
    result_message = "ประมวลผลไฟล์สำเร็จ!"
    return result_message, data_summary

def update_diagnosis_choices(cid):
    global diagnosis_mapping
    patient_row = uploaded_data[uploaded_data["cid"] == cid]
    if not patient_row.empty:
        icd10_code = patient_row['icd10'].iloc[0]
        diagnosis_description = diagnosis_mapping.get(icd10_code, "")
        return diagnosis_description  # Set as single choice and default value
    return ""

# Mappings for illness type and follow-up status
illness_type_mapping = {
    "ILL": "เจ็บป่วย",
    "ACC": "อุบัติเหตุ",
    "FU": "ผู้ป่วยนอกติดตามอาการหลังจาก Admit เจ็บป่วย",
    "ER": "อุบัติเหตุฉุกเฉินภายใน 24 ชม.",
    "HD": "ผู้ป่วยนอกล้างไต",
    "DEN": "ทันตกรรม"
}

follow_up_mapping = {
    "N": "เข้ารับการรักษาครั้งแรก",
    "Y": "เข้ารักษาแบบต่อเนื่อง"
}

def populate_patient_info(cid):
    global uploaded_data, uploaded_diag
    # ค้นหาผู้ป่วยตาม CID
    patient_row = uploaded_data[uploaded_data["cid"] == cid]
    if not patient_row.empty:
        patient_data = json.loads(patient_row.iloc[0]["patient"])  # Parse JSON
        icd10_code = patient_row.iloc[0]["icd10"]
        diagnosis_description = uploaded_diag[uploaded_diag['ICD-10 Code'] == icd10_code]['Description'].iloc[0] if not uploaded_diag[uploaded_diag['ICD-10 Code'] == icd10_code].empty else ""
        search_result = search_exact_match_score(icd10_code, 'internal-manageai-icd10-average_price')
        average_price = search_result[0]['_source']['total_average_price'] if search_result else 0.0
        billings = json.loads(patient_row.iloc[0]["billing"])
        order_items = json.loads(patient_row.iloc[0]["order_item"])

        # ตัวอย่าง: การแปลง Mapping
        illness_type_code = patient_row["illnes_type"].iloc[0]
        illness_type_mapped = illness_type_mapping.get(illness_type_code, "อุบัติเหตุ")
        visit_type_mapped = follow_up_mapping.get(patient_row["is_follow_up"].iloc[0], "Unknown")
        chief_complaint = patient_row["chief_complaint"].iloc[0]
        billing_table, total_net = get_billing_table_with_order_items(cid)

        # ตรวจสอบว่าค่าของ illness_type ถูกส่งไปที่ฟิลด์ที่เหมาะสม
        return (
            patient_data.get("title_name", ""),
            patient_data.get("first_name", ""),
            patient_data.get("last_name", ""),
            patient_data.get("hn", ""),
            patient_data.get("dob", ""),
            illness_type_mapped,  # ส่งไปที่ช่อง TextBox ของ treatment_type
            visit_type_mapped,
            chief_complaint,
            diagnosis_description.strip(),
            average_price,
            billings,
            order_items,
            billing_table.values.tolist(),
            total_net,
            cid
        )
    else:
        return "", "", "", "", "", "", "", "", "", "", "", "", 0.0, [], [], ""

def update_diagnosis_choices(cid):
    patient_row = uploaded_data[uploaded_data["cid"] == cid]
    if not patient_row.empty:
        icd10_code = patient_row['icd10'].iloc[0]
        diagnosis_description = diagnosis_mapping.get(icd10_code, "")
        return diagnosis_description  # Set as single choice and default value
    return ""

def get_simb_code(bill_description):
    description_to_simb = dict(zip(uploaded_simb['Description'], uploaded_simb['SIMB_1']))
    return description_to_simb.get(bill_description, "")

def get_billing_table_with_order_items(cid):
    patient_row = uploaded_data[uploaded_data["cid"] == cid]
    if not patient_row.empty:
        billing_data = json.loads(patient_row.iloc[0]["billing"])
        order_items_data = json.loads(patient_row.iloc[0]["order_item"])

        billing_df = pd.DataFrame(billing_data)
        order_items_df = pd.DataFrame(order_items_data)

        all_rows = []

        for _, billing_row in billing_df.iterrows():
            matching_items = order_items_df[order_items_df['simb_billing_code'] == billing_row['simb_billing_code']]

            # Add a row for the billing category
            all_rows.append({
                "ลำดับ": len(all_rows) + 1,
                "หมวดหมู่ค่าบริการ": f"{billing_row['simb_billing_code']} {billing_row['local_billing_name']}",
                "รายการ": "",
                "จำนวน": billing_row.get('item_amount', ""),  # Use get to safely access the key
                "จำนวนเงิน(ก่อนหักส่วนลด)": float(billing_row['billing_initial']),
                "ส่วนลด": float(billing_row['billing_discount']),
                "จำนวนเงิน(หลังหักส่วนลด)": float(billing_row['billing_net_amount']),
                "Billing Code": "",
                "SIMB": billing_row['simb_billing_code']
            })

            # Add rows for each item under this category
            for _, item in matching_items.iterrows():
                all_rows.append({
                    "ลำดับ": "",
                    "หมวดหมู่ค่าบริการ": item['item_id'],
                    "รายการ": item['item_name'],
                    "จำนวน": item['item_amount'],  # Access item_amount from order_items_df
                    "จำนวนเงิน(ก่อนหักส่วนลด)": float(item['initial']),
                    "ส่วนลด": float(item['discount']),
                    "จำนวนเงิน(หลังหักส่วนลด)": float(item['net_amount']),
                    "Billing Code": billing_row['local_billing_code'],
                    "SIMB": billing_row['simb_billing_code']
                })

        # Summation of numeric columns
        total_initial = billing_df['billing_initial'].astype(float).sum()
        total_discount = billing_df['billing_discount'].astype(float).sum()
        total_net = billing_df['billing_net_amount'].astype(float).sum()

        all_rows.append({
            "ลำดับ": "",
            "หมวดหมู่ค่าบริการ": "รวม",
            "รายการ": "",
            "จำนวน": "",
            "จำนวนเงิน(ก่อนหักส่วนลด)": total_initial,
            "ส่วนลด": total_discount,
            "จำนวนเงิน(หลังหักส่วนลด)": total_net,
            "Billing Code": "",
            "SIMB": ""
        })

        # Grand total row
        all_rows.append({
            "ลำดับ": "",
            "หมวดหมู่ค่าบริการ": "รวมค่ารักษาพยาบาลทั้งหมด",
            "รายการ": "",
            "จำนวน": "",
            "จำนวนเงิน(ก่อนหักส่วนลด)": total_initial,
            "ส่วนลด": total_discount,
            "จำนวนเงิน(หลังหักส่วนลด)": total_net,
            "Billing Code": "",
            "SIMB": ""
        })

        return pd.DataFrame(all_rows), total_net  # Return both DataFrame and total_net

    return pd.DataFrame(columns=["ลำดับ", "หมวดหมู่ค่าบริการ", "รายการ", "จำนวน", "จำนวนเงิน(ก่อนหักส่วนลด)", "ส่วนลด", "จำนวนเงิน(หลังหักส่วนลด)", "Billing Code", "SIMB"]), 0.0

def update_billing_table_with_order_items(cid):
    billing_df, total_net = get_billing_table_with_order_items(cid)
    return billing_df.values.tolist(), total_net

def calculate_recommended_claim(total_net, remaining_opd_year, total_average_price):
    total_net = float(total_net) if isinstance(total_net, str) else total_net
    remaining_opd_year = float(remaining_opd_year) if isinstance(remaining_opd_year, str) else remaining_opd_year
    total_average_price = float(total_average_price) if isinstance(total_average_price, str) else total_average_price
    if remaining_opd_year >= total_average_price:
        recommended_claim = total_average_price - total_net
    else:
        recommended_claim = remaining_opd_year - total_net

    return recommended_claim

def recommend_drugs_for_patient(patient_icd10_code):
    output = search_similar_vectors(patient_icd10_code, 'internal-manageai-icd10-order')
    if output is not None:
        filtered_output = output[output['simb_billing_code'] == '1.1.1(3)']
        sorted_output = filtered_output.sort_values(by=['order_count'], ascending=False)
        # top = sorted_output.drop(columns=['order_count','icd10','range']).head(10)
        top = sorted_output.drop(columns=['order_count','icd10','range'])
        return top
    else:
        return "No results found for the given ICD10 code."

def handle_form(
    id_card, title, first_name, last_name, patient_id, birth_date, remaining_opd_year,
    treatment_type, visit_type, chief_complaint, diag_type, diagnosis, total_net
):
    total_net_float = float(total_net)
    remaining_opd_year_float = float(remaining_opd_year) if remaining_opd_year else 0.0
    icd10_code = diagnosis.split(" ")[0] if diagnosis else None
    birth_date = datetime.strptime(birth_date, '%Y-%m-%d')

    # Calculate recommended claim
    output = search_exact_match_score(icd10_code, 'internal-manageai-icd10-average_price')
    if output:
        average_price = float(output[0]['_source'].get('total_average_price', 0.0))
        max_claimable = max(0, average_price - total_net_float)
        additional_claim = min(max_claimable, remaining_opd_year_float - total_net_float) if max_claimable > 0 else 0.0
        recommendation_adjustment = f"{'+ ' if additional_claim >= 0 else '- '}{abs(additional_claim):.2f}"
        recommended_claim = total_net_float + additional_claim
        claim_message = f"{recommended_claim:.2f}"
    else:
        recommendation_adjustment = "ไม่สามารถคำนวณได้"
        claim_message = "ไม่มีข้อมูลราคากลางสำหรับการรักษานี้"

    # Search recommended drugs for patient
    recommended_drugs = recommend_drugs_for_patient(icd10_code)
    if isinstance(recommended_drugs, pd.DataFrame):
        recommended_drugs_list = recommended_drugs.values.tolist()
    else:
        recommended_drugs_list = []  # Empty if no results

    return recommendation_adjustment, claim_message, recommended_drugs_list

class InMemoryCache:
    def __init__(self):
        self.cache = {}

    def get(self, key):
        return self.cache.get(key)

    def set(self, key, value):
        self.cache[key] = value

    def clear(self):
        self.cache.clear()


class CachingInferenceClient:
    def __init__(self, base_url, username, password):
        self.base_url = base_url
        self.auth = (username, password)
        self.cache = InMemoryCache()

    def text_generation(self, input_text, params):
        logging.info(f"Input Text: {input_text}")
        logging.info(f"Params: {params}")

        cached_result = self.cache.get(input_text)
        if cached_result:
            logging.info("Cache hit for input text.")
            return cached_result

        try:
            response = requests.post(
                # url=f"{self.base_url}",
                url=f"{self.base_url}/generate",

                data=json.dumps({"inputs": input_text, "parameters": params}),
                headers={'Content-Type': 'application/json'},
                auth=self.auth
            )
            logging.info(f"API Response Status Code: {response.status_code}")
            logging.info(f"API Response Text: {response.text}")

            response.raise_for_status()
            result = response.json()
            self.cache.set(input_text, result)
            return result
        except requests.exceptions.RequestException as e:
            logging.error(f"Error during API call: {e}")
            raise



# Logging setup
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

model_param = {
    "best_of": 1,
    "frequency_penalty": 1.3,
    "max_new_tokens": 300,
    "repetition_penalty": 1.2,
    "temperature": 0.2,
    "top_k": 10,
    "top_n_tokens": 5,
    "top_p": 0.95,
}


def generate_prompt2(text_inputs, prompt_template, model_params):
    # Replace placeholders in the template with the provided inputs
    formatted_prompt = prompt_template.format(
        icd10=text_inputs.get("icd10", "No ICD-10 Code Provided"),
        previous_drug=text_inputs.get("previous_drug", "No Current Drug Provided"),
        suggested_drug=text_inputs.get("suggested_drug", "No Suggested Drugs Provided"),
    )
    return {
        "input": formatted_prompt,
        "params": model_params
    }


def generate_summary_response2(prompt, client, retries=3):
    allowed_params = ["best_of", "max_new_tokens", "temperature", "top_k", "top_p"]  # Adjust based on API specs
    filtered_params = {k: v for k, v in prompt['params'].items() if k in allowed_params}

    for attempt in range(retries):
        try:
            logging.info(f"Attempt {attempt + 1}: Sending request to the API.")
            response = client.text_generation(prompt['input'], filtered_params)
            logging.info("API response received successfully.")
            return response
        except requests.exceptions.HTTPError as e:
            logging.error(f"HTTPError occurred: {e}")
            if "401" in str(e):
                logging.error("Unauthorized access. Check your username and password.")
                raise
            elif "Model is overloaded" in str(e):
                backoff_time = 2 ** attempt  # Exponential backoff
                logging.warning(f"Model is overloaded. Retrying after {backoff_time} seconds...")
                time.sleep(backoff_time)
            else:
                raise
        except Exception as e:
            logging.error(f"Unexpected error occurred: {e}")
            raise
    raise Exception("Max retries exceeded")

default_prompt = """
YOU ARE A WORLD-CLASS PHARMACIST AND MEDICAL EXPERT, RENOWNED FOR YOUR EXPERTISE IN OPTIMIZING DRUG REGIMENS TO IMPROVE PATIENT OUTCOMES. YOUR TASK IS TO ANALYZE A GIVEN ICD-10 DIAGNOSIS, EVALUATE THE CURRENT DRUG REGIMEN, AND RECOMMEND ONE OR MORE ADDITIONAL DRUGS FROM A PROVIDED LIST. YOU MUST SELECT DRUGS THAT IMPROVE CLINICAL OUTCOMES AND ALIGN WITH TREATMENT GUIDELINES FOR THE CONDITION WITHOUT OVERLOADING THE PATIENT WITH UNNECESSARY OR POTENTIALLY DANGEROUS EXCESS MEDICATION.

### INSTRUCTIONS ###

1. **UNDERSTAND THE DIAGNOSIS AND TREATMENT GUIDELINES**:
   - REVIEW the ICD-10 diagnosis to understand the condition and standard treatment guidelines.
   - DETERMINE if the current drug regimen adequately addresses the condition and IDENTIFY any gaps or potential for improvement.

2. **ASSESS SUGGESTED DRUG OPTIONS**:
   - ANALYZE the list of suggested drugs, including their indications.
   - SELECT additional drug(s) that enhance the effectiveness of the current regimen.
   - CHOOSE drugs that are NECESSARY and BENEFICIAL, but BE MINDFUL OF the total number of medications to AVOID OVERLOADING THE PATIENT.

3. **CONSIDER DOSAGE SAFETY**:
   - ASSESS the overall medication load, ensuring that adding more drugs does not create a risk of overdose or adverse effects due to excessive polypharmacy.
   - REVIEW the patient's existing drug regimen to ENSURE that the combined effects of all medications are safe and clinically appropriate.

4. **ENSURE SAFETY AND CLINICAL EFFICACY**:
   - CHECK for potential drug-drug interactions or contraindications with the current regimen.
   - VERIFY the safety and efficacy of the selected drug(s) based on clinical guidelines and evidence.

5. **JUSTIFY YOUR RECOMMENDATION**:
   - PROVIDE a clear rationale for each selected drug, explaining how it improves patient outcomes, aligns with clinical guidelines, and adds meaningful value to the regimen.
   - AVOID recommending excessive medication; ensure the selected drugs offer significant clinical benefits without risking overdose or unnecessary complexity.

6. **HANDLE EXCEPTIONS**:
   - IF no suitable additional drugs are identified, PROVIDE alternative strategies, such as adjusting the current regimen or focusing on the most critical aspects of the patient's treatment.

7. **FINALIZE AND SUMMARIZE**:
   - SUMMARIZE the selected drug(s) along with a concise rationale that emphasizes clinical effectiveness, safety, and dosage appropriateness.

### WHAT NOT TO DO ###

- DO NOT RECOMMEND DRUGS THAT DO NOT ALIGN WITH TREATMENT GUIDELINES FOR THE DIAGNOSIS.
- DO NOT IGNORE POTENTIAL DRUG INTERACTIONS OR CONTRAINDICATIONS.
- DO NOT ADD UNNECESSARY MEDICATION THAT DOES NOT SIGNIFICANTLY IMPROVE OUTCOMES.
- DO NOT OVERLOAD THE REGIMEN WITH TOO MANY MEDICATIONS OR RISK OVERDOSING THE PATIENT.
- DO NOT PROVIDE A RECOMMENDATION WITHOUT A CLEAR AND EVIDENCE-BASED RATIONALE.

### TEMPLATE ###

ICD-10 Diagnosis: {icd10}
Current Drug Regimen: {previous_drug}
Suggested Drug Options: {suggested_drug}

### TASK ###

BASED ON the ICD-10 diagnosis "{icd10}", the current drug regimen "{previous_drug}", and the suggested drugs "{suggested_drug}", RECOMMEND one or more additional drug(s) that MAXIMIZE clinical outcomes without overloading the regimen. ENSURE that each selected drug adds meaningful value based on clinical guidelines and patient needs, and that the overall medication load remains safe without risk of overdose. JUSTIFY your choice with clear reasoning that considers clinical effectiveness, safety, and dosage appropriateness.

### EXAMPLES ###

#### Example 1 ####
ICD-10 Diagnosis: M54.5 (Low back pain)
Current Drug Regimen: ['paracetamol 500 mg']
Suggested Drug Options: ['PERSKINDOL SPRAY', 'MYONAL 50 MG']

**Response**:
ANSWER: ['PERSKINDOL SPRAY', 'MYONAL 50 MG']
RATIONALE: ["PERSKINDOL SPRAY บรรเทาอาการปวด is added to provide topical pain relief, which is beneficial for superficial injuries and can reduce the need for systemic pain medications. This aligns with guidelines for managing superficial injuries by providing localized treatment. MYONAL 50 MG ยาคลายกล้ามเนื้อ is selected to address potential muscle spasms associated with the injury, enhancing the patient's comfort and mobility. Both additions are aimed at improving symptom management without significantly increasing the risk of adverse effects or interactions with the current regimen."]

#### Example 2 ####
ICD-10 Diagnosis: G44.1 (Vascular headache, including migraine)
Current Drug Regimen: ['ibuprofen 400 mg PRN']
Suggested Drug Options: ['sumatriptan 50 mg', 'naproxen 500 mg']

**Response**:
ANSWER: ['sumatriptan 50 mg']
RATIONALE: ["Sumatriptan 50 mg is selected as it is a first-line treatment for acute migraines, targeting the underlying pathophysiology of vascular headaches by activating serotonin receptors and reducing inflammation. It complements the current regimen by providing a more targeted treatment option than ibuprofen. Naproxen is not selected in this case to avoid unnecessary overlap with the current NSAID therapy (ibuprofen), which already addresses inflammation. This ensures effective migraine relief while minimizing polypharmacy risks."]

### OUTPUT FORMAT ###
ANSWER: ['selected_drug(s)']
RATIONALE: ["Justification for each selected drug, including how it enhances the treatment, aligns with guidelines, avoids overdose, and minimizes unnecessary medication overload."]
**DO NOT PROVIDE ANY INFORMATION OUTSIDE THIS FORMAT.**
"""

# Function to filter billing items
def filter_billing_items(billing_data):
    # # Ensure billing_data is a DataFrame
    if not isinstance(billing_data, pd.DataFrame):
        print("Error: Expected a DataFrame, got:", type(billing_data))
        return []

    if billing_data.empty:
        return []

    # Filter rows where "SIMB" is "1.1.1(3)" and "รายการ" is not NaN or empty
    filtered_data = billing_data[
        (billing_data["SIMB"] == "1.1.1(3)") &
        (billing_data["รายการ"].notna()) &
        (billing_data["รายการ"].str.strip() != "")
    ]

    return filtered_data["รายการ"].tolist()


# Function to filter recommended drugs
def filter_recommended_drugs(drug_data):
    if not isinstance(drug_data, pd.DataFrame):
        print("Error: Expected a DataFrame, got:", type(drug_data))
        return []

    if drug_data.empty:
        return []

    # Filter rows where "SIMB" matches the condition
    filtered_data = drug_data[drug_data["SIMB"] == "1.1.1(3)"]
    return filtered_data["Item Name"].tolist()


# Function to prepare LLM inputs
def prepare_llm_inputs(billing_data, drug_data, icd10_input):
    # Debugging output
    print(billing_data)
    print('\n')
    print(drug_data)
    return {
        "icd10": icd10_input,
        "previous_drug": billing_data,
        "suggested_drug": drug_data
    }


# Function to extract answer and rationale from response
def extract_answer_and_rationale(text):
    """
    Extracts the ANSWER and RATIONALE sections from a given text.
    """
    answer_pattern = r"ANSWER:\s*\[(.*?)\]"
    rationale_pattern = r"RATIONALE:\s*\[(.*?)\]"

    answer_match = re.search(answer_pattern, text, re.DOTALL)
    rationale_match = re.search(rationale_pattern, text, re.DOTALL)

    return {
        "ANSWER": answer_match.group(1) if answer_match else None,
        "RATIONALE": rationale_match.group(1) if rationale_match else None
    }

def get_llm_response(billing_data, drug_data, icd10_input, default_prompt, model_param, username_llm, passwords_llm):
    # Prepare the inputs
    text_inputs = prepare_llm_inputs(billing_data, drug_data, icd10_input)

    # Convert text_inputs to JSON format for display in Markdown
    display_text_inputs = json.dumps(text_inputs, indent=4, ensure_ascii=False)

    # Generate the prompt based on the prepared inputs
    prompt = generate_prompt2(text_inputs, default_prompt, model_param)

    # Create the client
    client = CachingInferenceClient('https://ai-api.manageai.co.th/llm-model-03', username_llm, passwords_llm)

    # Generate response using the client
    response = generate_summary_response2(prompt, client)

    # Extract ANSWER and RATIONALE using the provided function
    extracted_response = extract_answer_and_rationale(response['generated_text'])
    response_gen_text = json.dumps(response['generated_text'], indent=4, ensure_ascii=False)

    answer = extracted_response['ANSWER']
    rationale = extracted_response['RATIONALE']

    print(display_text_inputs)
    print('--------------------------------------------------------------###')

    print(response['generated_text'])
    print('--------------------------------------------------------------###')
    response = json.dumps(response, indent=4, ensure_ascii=False)
    return response_gen_text, answer, rationale

def get_patient_data_as_json(id_card, title, first_name, last_name, patient_id, birth_date, treatment_type, visit_type,
                             chief_complaint, diagnosis, billings, order_items, total_net, remaining_opd_year, final_result, recommendation_adjustment
):
    patient_data = {
        "id_card": id_card,
        "title": title,
        "first_name": first_name,
        "last_name": last_name,
        "patient_id": patient_id,
        "birth_date": birth_date,
        "treatment_type": treatment_type,
        "visit_type": visit_type,
        "chief_complaint": chief_complaint,
        "diagnosis": diagnosis,
        "billings": billings,
        "order_items": order_items,
        "remaining_opd_year": remaining_opd_year,
        "total_net": total_net,
        "final_result": final_result,
        "recommendation_adjustment": recommendation_adjustment
    }
    json_output = json.dumps(patient_data, indent=4, ensure_ascii=False)
    return json_output

custom_css = """
.red-border {
    border: 2px solid red !important;
    border-radius: 5px;
    padding: 5px;
}
"""

### Ui
with gr.Blocks() as demo:
    with gr.Tab("Upload Data Files"):
        gr.Markdown("### อัปโหลดไฟล์ CSV ที่ใช้ในการวิเคราะห์")

        # Input for CSV files
        with gr.Row():
            data_file = gr.File(label="อัปโหลดไฟล์ data.csv")
            simb_file = gr.File(label="อัปโหลดไฟล์ simblist.csv")
            diag_file = gr.File(label="อัปโหลดไฟล์ diaglist.csv")

        # Display message after uploading files
        upload_button = gr.Button("อัปโหลดไฟล์")
        upload_result = gr.Textbox(label="ผลลัพธ์การอัปโหลดไฟล์", interactive=False)

    with gr.Tab("Patient and Claim Information"):
        gr.Markdown("## ข้อมูลผู้ป่วยนอก")
        id_card = gr.Textbox(label="เลขบัตรประชาชน *", placeholder="กรอกเลขบัตรประชาชนของผู้ป่วย", elem_classes="red-border")
        sample_cid_buttons = gr.Radio(label="เลือกตัวอย่าง CID", choices=[], type="value")
        with gr.Row():
            title = gr.Dropdown(
                label="คำนำหน้า",
                choices=["คุณ", "นาง", "นางสาว", "นาย", "Mr", "Ms", "Miss", "Mrs", "เด็กชาย", "เด็กหญิง"]
            )
            first_name = gr.Textbox(label="ชื่อ")
            last_name = gr.Textbox(label="นามสกุล")
        with gr.Row():
            patient_id = gr.Textbox(label="รหัสผู้ป่วย (HN)")
            birth_date = gr.Textbox(label="วัน/เดือน/ปีเกิด (ค.ศ.)", placeholder="yyyy-mm-dd")

        gr.Markdown("### ข้อมูลส่งเคลม")

        with gr.Row():
            treatment_type = gr.Dropdown(
                label="ประเภทการรักษา",
                value="",  # ค่าที่ได้จากการแมพ
                choices=list(illness_type_mapping.values())
                )
            visit_type = gr.Radio(
                label="ประเภทการเข้ารักษา",
                choices=list(follow_up_mapping.values())
            )
        chief_complaint = gr.Textbox(label="อาการสำคัญที่เข้ามาโรงพยาบาล", value="")

        gr.Markdown("#### Diagnosis Type")
        with gr.Row():
            diag_type = gr.Dropdown(
                label="Diagnosis Type",
                choices=["โรคหลักที่ให้การรักษา (PP)", "โรคประจำตัว (UD)", "การวินิจฉัยร่วม (CM)", "โรคแทรก (CP)", "โรคอื่นๆ (OT)"],
                value='โรคหลักที่ให้การรักษา (PP)'
            )
            diagnosis = gr.Dropdown(
                label="Diagnosis (ICD10) *",
                choices=list(diagnosis_mapping.values()) if diagnosis_mapping else [],
                value=""
            )

        gr.Markdown("#### รายการค่ารักษาพยาบาล")
        billing_table = gr.DataFrame(
            headers=["ลำดับ", "หมวดหมู่ค่าบริการ", "รายการ", "จำนวน", "จำนวนเงิน(ก่อนหักส่วนลด)", "ส่วนลด", "จำนวนเงิน(หลังหักส่วนลด)", "Billing Code", "SIMB"],
            interactive=True
        )
        with gr.Row():
            billings = gr.Textbox(label="รายการบิล", interactive=False, visible=False)
            order_items = gr.Textbox(label="รายการยา", interactive=False, visible=False)

    with gr.Tab("Recommend Claim"):
        gr.Markdown("#### จำนวนเงินที่สามารถเบิกได้")
        with gr.Row():
            total_net = gr.Number(label="รวมค่ารักษาพยาบาลในการส่งยอดเคลมครั้งนี้ (บาท) ", interactive=False)
            average_price = gr.Number(label="จำนวนเบิกได้สูงสุดสำหรับโรคนี้ (บาท)", interactive=False)

        id_card.change(
            fn=update_billing_table_with_order_items,
            inputs=[id_card],
            outputs=[billing_table, total_net]
        )

        remaining_opd_year = gr.Textbox(label="จำนวนเงิน OPD คงเหลือ *", placeholder="กรอกจำนวนเงิน OPD คงเหลือ", elem_classes="red-border")

        with gr.Row(elem_classes="red-border"):
            recommendation_adjustment = gr.Textbox(label="แนะนำให้เบิกเพิ่ม(+)/ลด(-) (บาท)", interactive=False, visible=True)
            final_result = gr.Textbox(label="สามารถแนะนำให้เบิกได้สูงสุดสำหรับครั้งนี้ (บาท)", interactive=False, visible=True)  # เพิ่มช่องนี้

        id_card.change(
            fn=populate_patient_info,
            inputs=[id_card],
            outputs=[title, first_name, last_name, patient_id, birth_date, treatment_type, visit_type, chief_complaint, diagnosis, average_price, billings, order_items, billing_table, total_net, id_card]
        )

        id_card.change(
            fn=update_diagnosis_choices,
            inputs=[id_card],
            outputs=[diagnosis]
        )

        sample_cid_buttons.change(
            fn=populate_patient_info,
            inputs=[sample_cid_buttons],
            outputs=[title, first_name, last_name, patient_id, birth_date, treatment_type, visit_type, chief_complaint, diagnosis, average_price, billings, order_items, billing_table, total_net, id_card]
        )

        upload_button.click(
            fn=load_files,
            inputs=[data_file, simb_file, diag_file],
            outputs=[upload_result, sample_cid_buttons, diagnosis]
        )

        gr.Markdown("##### รายการยาที่มักใช้ในผู้ป่วยโรคเดียวกัน")
        recommended_drugs_table = gr.DataFrame(
            headers=["SIMB", "Billing Name", "Item ID", "Item Name", "Average Price"], interactive=False)

        submit_button = gr.Button("Submit")
        submit_button.click(
            fn=handle_form,
            inputs=[id_card, title, first_name, last_name, patient_id, birth_date, remaining_opd_year,
                    treatment_type, visit_type, chief_complaint, diag_type, diagnosis, total_net],
            outputs=[recommendation_adjustment, final_result, recommended_drugs_table]
        )

        remaining_opd_year.submit(
            fn=handle_form,
            inputs=[id_card, title, first_name, last_name, patient_id, birth_date, remaining_opd_year,
                    treatment_type, visit_type, chief_complaint, diag_type, diagnosis, total_net],
            outputs=[recommendation_adjustment, final_result, recommended_drugs_table]
        )
    with gr.Tab("Filtered Results"):
        billing_items_result = gr.Textbox(label="Billing Items (1.1.1(3))", interactive=False)
        recommended_drugs_result = gr.Textbox(label="Recommended Drugs (1.1.1(3))", interactive=False)

        # Button to process and filter (move it here)
        filter_button = gr.Button("Filter Data")

        # Logic: Update filtered arrays
        filter_button.click(
            fn=lambda billing, drugs: (filter_billing_items(billing), filter_recommended_drugs(drugs)),
            inputs=[billing_table, recommended_drugs_table],
            outputs=[billing_items_result, recommended_drugs_result]
        )

        # Input_format  = gr.Markdown(label="Input_format Results")
        llm_output = gr.Markdown(label="LLM Suggestion Results")
        answer_output_box = gr.Textbox(label="ANSWER", placeholder="Extracted ANSWER will appear here")
        rationale_output_box = gr.Textbox(label="RATIONALE", placeholder="Extracted RATIONALE will appear here")

        # Define the button click function
        llm_btn = gr.Button("Suggest_By_LLM")
        llm_btn.click(
            get_llm_response,
            inputs=[billing_items_result, recommended_drugs_result, diagnosis, gr.State(default_prompt), gr.State(model_param), gr.State(username_llm), gr.State(passwords_llm)],
            outputs=[llm_output, answer_output_box, rationale_output_box]
        )

    with gr.Tab("Get Patient Data"):
        with gr.Row():
            json_output = gr.Textbox(label="Patient Data", interactive=False)
        json_button = gr.Button("Get Data")

        json_button.click(
            fn=get_patient_data_as_json,
            inputs=[id_card, title, first_name, last_name, patient_id, birth_date,treatment_type, visit_type,
                    chief_complaint, diagnosis, billings, order_items, total_net, remaining_opd_year, final_result, recommendation_adjustment],
            outputs=json_output
        )

demo.launch(debug=True, share=True)

/usr/local/lib/python3.10/dist-packages/gradio/components/dropdown.py:226: UserWarning: The value passed into gr.Dropdown() is not in the list of choices. Please update the list of choices to include:  or set allow_custom_value=True.
  warnings.warn(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://8466c1cd86b81391f2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Sample CIDs: ['1309902530511', '1560400051072', '1100501777111', '1104700019163', '3710600631531']


<ipython-input-10-36d44a9a2353>:70: DeprecationWarning: Received 'timeout' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  response = es.search(index=index_name, body=search_query, timeout='120s')
<ipython-input-10-36d44a9a2353>:70: DeprecationWarning: Received 'timeout' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  response = es.search(index=index_name, body=search_query, timeout='120s')
<ipython-input-10-36d44a9a2353>:70: DeprecationWarning: Received 'timeout' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  response = es.search(index=index_name, body=search_query, timeout='120s')
<ipython-i





{
    "icd10": "J069 การติดเชื้อทางเดินหายใจส่วนบนเฉียบพลัน ไม่ระบุรายละเอียด (Acute upper respiratory infection, unspecified [ URI ])",
    "previous_drug": "",
    "suggested_drug": ""
}
--------------------------------------------------------------###
**ENSURE THAT THE RESPONSE IS CONCISE AND FOCUSED ON THE CLINICAL RATIONALE.**assistant

ICD-10 Diagnosis: J069 การติดเชื้อทางเดินหายใจส่วนบนเฉียบพลัน ไม่ระบุรายละเอียด (Acute upper respiratory infection, unspecified [ URI ])
Current Drug Regimen: 
Suggested Drug Options: ['paracetamol 500 mg', 'loratadine 10 mg', 'ambroxol 30 mg']

ANSWER: ['paracetamol 500 mg', 'ambroxol 30 mg']
RATIONALE: ["Paracetamol 500 mg is added to provide relief from fever and headache associated with URI, aligning with guidelines for symptomatic management. Ambroxol 30 mg is selected to help thin mucus, making it easier to cough up, thus providing relief from congestion. Loratadine is not selected as it's primarily used for allergies, which isn't the pri

In [ ]:
{
    "id_card": "1309902530511",
    "title": "นาย",
    "first_name": "คณิศร",
    "last_name": "ทองดีบุตร",
    "patient_id": "6609393",
    "birth_date": "1997-12-22",
    "treatment_type": "เจ็บป่วย",
    "visit_type": "เข้ารับการรักษาครั้งแรก",
    "chief_complaint": "ปวดศีรษะ เจ็บคอSpo2=97%",
    "diagnosis": "J069 การติดเชื้อทางเดินหายใจส่วนบนเฉียบพลัน ไม่ระบุรายละเอียด (Acute upper respiratory infection, unspecified [ URI ])",
    "billings": "[{'billing_initial': '712.00', 'billing_discount': '71.00', 'simb_billing_code': '1.1.1(3)', 'billing_net_amount': '641.00', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'billing_initial': '100', 'billing_discount': '0', 'simb_billing_code': '1.1.12', 'billing_net_amount': '100.00', 'local_billing_code': 'SER', 'local_billing_name': 'Nursing and Midwifery Charge   ค่าบริการพยาบาล'}, {'billing_initial': '150', 'billing_discount': '0', 'simb_billing_code': '1.1.14(2)', 'billing_net_amount': '150.00', 'local_billing_code': 'SER2', 'local_billing_name': 'Other Medical Service Charge ค่าบริการทางการแพทย์'}, {'billing_initial': '600', 'billing_discount': '0', 'simb_billing_code': '1.2.1', 'billing_net_amount': '600.00', 'local_billing_code': 'DFO', 'local_billing_name': 'Physician Evaluation and Management Services'}]",
    "order_items": "[{'initial': '50', 'item_id': 'DRUG', 'discount': '0', 'item_name': 'Ambroxol TAB 30  mg', 'net_amount': '50.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '300', 'item_id': 'DRUG', 'discount': '0', 'item_name': 'Claron TAB 500 mg ;', 'net_amount': '300.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '100', 'item_id': 'DRUG', 'discount': '0', 'item_name': 'Exib (Etoricoxib) TA', 'net_amount': '100.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '42', 'item_id': 'DRUG', 'discount': '0', 'item_name': 'Loratadine (Carinose', 'net_amount': '42.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '180', 'item_id': 'DRUG', 'discount': '71', 'item_name': 'Rotuss TAB 10/100 mg', 'net_amount': '109.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '40', 'item_id': 'DRUG', 'discount': '0', 'item_name': 'Tylenol TAB 500 mg', 'net_amount': '40.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '100', 'item_id': 'SER', 'discount': '0', 'item_name': 'ค่าบริการพยาบาล', 'net_amount': '100.00', 'item_amount': '1', 'simb_billing_code': '1.1.12', 'local_billing_code': 'SER', 'local_billing_name': 'Nursing and Midwifery Charge   ค่าบริการพยาบาล'}, {'initial': '150', 'item_id': 'SER2', 'discount': '0', 'item_name': 'ค่าบริการโรงพยาบาลผู', 'net_amount': '150.00', 'item_amount': '1', 'simb_billing_code': '1.1.14(2)', 'local_billing_code': 'SER2', 'local_billing_name': 'Other Medical Service Charge ค่าบริการทางการแพทย์'}, {'initial': '600', 'item_id': 'DFO', 'discount': '0', 'item_name': 'ค่าตรวจรักษาทั่วไป', 'net_amount': '600.00', 'item_amount': '1', 'simb_billing_code': '1.2.1', 'local_billing_code': 'DFO', 'local_billing_name': 'Physician Evaluation and Management Services'}]",
    "remaining_opd_year": "5000",
    "total_net": "1491.0",
    "final_result": "5000.00",
    "recommendation_adjustment": "+ 3509.00"
}